Запускаем ячейку, чтобы установить библиотеки и нужное для работы со Spark окружение.

In [1]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

ЋиЁЎЄ  ў бЁ­в ЄбЁбҐ Є®¬ ­¤л.


Инициализируем спарк-сессию. SparkSession — это, по сути, точка входа для работы с Apache Spark в программе-клиенте. После инициализации нам будут доступны базовые возможности фреймворка Spark для создания распределённых массивов данных RDD, DataFrame и DataSet.

In [3]:
from pyspark.sql import SparkSession

In [4]:


spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

KeyboardInterrupt: 

[Сайт, где можно найти всё или почти всё о синтаксисе PySpark](https://sparkbyexamples.com/pyspark/).

### Загрузка данных

[Сайт, откуда можно скачать датасет с квартирами](https://www.kaggle.com/datasets/mrdaniilak/russia-real-estate-2021?select=input_data.csv).

При запуске ячейки, расположенной ниже, всплывающее меню прокрутите до конца и нажмите «Разрешить».

In [ ]:
from google.colab import drive #Импортируем библиотеку для работы с Google Drive
drive.mount("/content/drive") #Монтируем Google-диск. После этой операции мы можем считывать файлы со своего Google-диска.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Чтобы следующая ячейка запустилась без ошибок, нужно сохранить датасет на свой Google-диск.

Загружаем датасет с недвижимостью в России с диска.

Для загрузки данных используем read.csv():

*  .option("delimiter",";") — указываем разделитель для считываемого CSV-файла.
*  .option("header",True) — читаем первую строку CSV как заголовки столбцов будущей таблицы.




In [ ]:
realty = spark.read.option("header",True).option("delimiter",";").csv("/content/drive/MyDrive/Colab Notebooks/input_data.csv") 
#Читаем наши данные. "/content/drive/MyDrive/Colab Notebooks/input_data.csv" — это путь к файлу с данными на диске. У вас путь может быть другой

In [ ]:
type(realty) #Смотрим тип нашей переменной

pyspark.sql.dataframe.DataFrame

Чтобы отобразить первые строки датафрейма, используем метод show(), который по умолчанию в PySpark показывает 20 строк.



In [ ]:
realty.show()

+----------+--------+-----+------+-----+-----+------------+----------+-----------+-------------+-----------+-----------+---------+---------+--------+
|      date|   price|level|levels|rooms| area|kitchen_area|   geo_lat|    geo_lon|building_type|object_type|postal_code|street_id|id_region|house_id|
+----------+--------+-----+------+-----+-----+------------+----------+-----------+-------------+-----------+-----------+---------+---------+--------+
|2021-01-01| 2451300|   15|    31|    1| 30.3|           0|56.7801124| 60.6993548|            0|          2|     620000|     null|       66| 1632918|
|2021-01-01| 1450000|    5|     5|    1|   33|           6|44.6081542| 40.1383814|            0|          0|     385000|     null|        1|    null|
|2021-01-01|10700000|    4|    13|    3|   85|          12|55.5400601| 37.7251124|            3|          0|     142701|   242543|       50|  681306|
|2021-01-01| 3100000|    3|     5|    3|   82|           9|44.6081542| 40.1383814|            0|    

In [ ]:
realty.show(5,8) #Показать первые 5 строк, по 8 символов на каждый столбец

+--------+--------+-----+------+-----+----+------------+--------+--------+-------------+-----------+-----------+---------+---------+--------+
|    date|   price|level|levels|rooms|area|kitchen_area| geo_lat| geo_lon|building_type|object_type|postal_code|street_id|id_region|house_id|
+--------+--------+-----+------+-----+----+------------+--------+--------+-------------+-----------+-----------+---------+---------+--------+
|2021-...| 2451300|   15|    31|    1|30.3|           0|56.78...|60.69...|            0|          2|     620000|     null|       66| 1632918|
|2021-...| 1450000|    5|     5|    1|  33|           6|44.60...|40.13...|            0|          0|     385000|     null|        1|    null|
|2021-...|10700000|    4|    13|    3|  85|          12|55.54...|37.72...|            3|          0|     142701|   242543|       50|  681306|
|2021-...| 3100000|    3|     5|    3|  82|           9|44.60...|40.13...|            0|          0|     385000|     null|        1|    null|
|2021-

Метод printSchema() показывает структуру таблицы (датафрейма): имена и тип столбцов.

In [ ]:
realty.printSchema()

root
 |-- date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- level: string (nullable = true)
 |-- levels: string (nullable = true)
 |-- rooms: string (nullable = true)
 |-- area: string (nullable = true)
 |-- kitchen_area: string (nullable = true)
 |-- geo_lat: string (nullable = true)
 |-- geo_lon: string (nullable = true)
 |-- building_type: string (nullable = true)
 |-- object_type: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- street_id: string (nullable = true)
 |-- id_region: string (nullable = true)
 |-- house_id: string (nullable = true)



Как видите, все столбцы нашего датафрейма — строкового типа.

### Предобработка данных

Поменяем тип столбцов датафрейма.

Это можно сделать разными способами, например:
*   с помощью метода select: your_dataframe.select(col("column_name").cast("new_type") ...)
*   через selectExpr: your_dataframe.selectExpr("CAST(culumn_name AS new_type) as column_name", ...)



In [ ]:
from pyspark.sql.functions import col,sum,avg,max,count,expr

realty_formatted = realty.select(
    col("level").cast("Int"),
    col("levels").cast("Int"),
    col("rooms").cast("Int"),
    col("area").cast("Double"),
    col("kitchen_area").cast("Double"),
    col("geo_lat").cast("Double"),
    col("geo_lon").cast("Double"),
    col("building_type").cast("Int"),
    col("object_type").cast("Int"),
    col("id_region").cast("Int"),
    col("price").cast("Double")
)
#Мы выбрали из датафрейма колонки level, levels, rooms, area, kitchen_area, geo_lat, geo_lon, building_type, object_type, object_type, id_region, price,
#одновременно изменив их тип при помощи оператора cast(). Те колонки, которые мы не перечислили внутри Select, не попали в новый датафрейм realty_formatted.



Рассмотрим структуру получившегося датафрейма.

In [ ]:
realty_formatted.printSchema()

root
 |-- level: integer (nullable = true)
 |-- levels: integer (nullable = true)
 |-- rooms: integer (nullable = true)
 |-- area: double (nullable = true)
 |-- kitchen_area: double (nullable = true)
 |-- geo_lat: double (nullable = true)
 |-- geo_lon: double (nullable = true)
 |-- building_type: integer (nullable = true)
 |-- object_type: integer (nullable = true)
 |-- id_region: integer (nullable = true)
 |-- price: double (nullable = true)



Посчитаем количество строк. Это можно сделать при помощи метода .count().

In [ ]:
realty_formatted.count()

11358150

Выберем квартиры, продаваемые в Московском регионе.

Фильтровать данные в датафрейме позволяет метод .filter() (аналог WHERE в SQL).

In [ ]:
realty_S0 = realty_formatted.filter(realty_formatted.id_region == 50)

Как видим, размер датафрейма значительно уменьшился.


In [ ]:
realty_S0.count()

754267

Отобразим первые строки датафрейма. Видим, что в нём есть квартиры с отрицательной площадью кухни.

In [ ]:
realty_S0.show()

+-----+------+-----+-----+------------+----------+----------+-------------+-----------+---------+---------+
|level|levels|rooms| area|kitchen_area|   geo_lat|   geo_lon|building_type|object_type|id_region|    price|
+-----+------+-----+-----+------------+----------+----------+-------------+-----------+---------+---------+
|    4|    13|    3| 85.0|        12.0|55.5400601|37.7251124|            3|          0|       50|   1.07E7|
|    5|    17|   -1|32.44|         0.0|55.4624497| 37.700041|            0|          2|       50|3214804.0|
|    3|    17|    2| 62.4|         9.7|55.6905542|37.8635456|            0|          0|       50|   1.11E7|
|    2|    17|    1|36.99|       11.72|55.5639247|37.8255637|            4|          2|       50|4296056.0|
|    5|     5|    3| 66.1|         9.0|55.9072587|38.0579851|            2|          0|       50|3899000.0|
|    4|     9|    3| 52.4|         6.0|55.7496117|38.0076555|            2|          0|       50|6000000.0|
|    7|    17|    1| 39.0|  

Посмотрим, есть ли в датафрейме другие квартиры, площадь кухни которых отрицательная.

Сгруппируем квартиры с отрицательной площадью и выясним, в скольких из них параметры кухни имеют «неправильное» значение.

Код ниже эквивалентен следующему SQL-запросу: SELECT kitchen_area, count(price) as count_items 
FROM realty_S0 WHERE kitchen_area < 0
GROUP BY kitchen_area

In [ ]:
realty_S0.filter(col("kitchen_area") < 0).groupBy("kitchen_area").agg(count("price").alias("count_items")).show()

+------------+-----------+
|kitchen_area|count_items|
+------------+-----------+
|      -100.0|      56262|
+------------+-----------+



Выберем только те квартиры, площадь кухни которых ≥ 0.

In [ ]:
realty_S1 = realty_S0.filter(col("kitchen_area") >= 0)

Снова посчитаем количество строк в датасете.

In [ ]:
realty_S1.count()

698005

Перейдём к feature-инженирингу.

Очевидно, что координаты дома, где находится квартира, не слишком коррелируют с ценой. На цену квартиры, как правило, влияет расстояние до центра города.

Возьмём координаты центра Москвы и коэффициенты перевода градусов широты и долготы (на широтах в 55 градусов в Северном полушарии) в километры.

In [ ]:
Moscow_lat = 55.753995
Moscow_lon = 37.614069

lat_to_km = 113.3 #Количество километров в одном градусе долготы
lon_to_km = 64.0 #Количество километров в одном градусе широты

Напишем функцию  coord_to_distance, которая будет брать на вход координаты дома, а выдавать расстояние от дома до центра Москвы.

In [ ]:
def coord_to_distance(latitude,longitude):
    distance = (((latitude - Moscow_lat)*lat_to_km)**2 + ((longitude - Moscow_lon)*lon_to_km)**2) ** 0.5
    return distance

Применим нашу функцию к столбцам geo_lat и geo_lon. Создадим новый — distance_to_center, а названные столбцы удалим.

Новый столбец в датафрейме можно создать с помощью метода withColumn(). Удалять стоблцы можно при помощи drop().

In [ ]:
realty_S2 = realty_S1.withColumn("distance_to_center",coord_to_distance(col("geo_lat"),col("geo_lon"))).drop("geo_lat","geo_lon","id_region")
#Подробное описание: наш_датафрейм.withColumn("имя_новой_колонки",Операции над старыми колонками для получения новой).drop("столбец_который_мы_хотим_удалить1","столбец_который_мы_хотим_удалить2",...)

Создадим два бинарных признака:


*   столбец not_first_level, принимающий 1, если квартира не на первом, и 0, если квартира на первом этаже;
*   столбец not_last_level, принимающий 1, если квартира не на последнем, и 0, если квартира на последнем этаже.

Чтобы получить значения новых столбцов, используем оператор when:

when(условие, значение, если true).otherwise(значение, если false).



In [ ]:
from pyspark.sql.functions import when  

realty_S3 = realty_S2.withColumn("not_first_level",when(col("level") == 1,0).otherwise(1)).withColumn("not_last_level",when(col("level") == col("levels"),0).otherwise(1))
#Описание работы when: when(условие, значение, если true).otherwise(значение, если false)

In [ ]:
realty_S3.show(5)

+-----+------+-----+-----+------------+-------------+-----------+---------+------------------+---------------+--------------+
|level|levels|rooms| area|kitchen_area|building_type|object_type|    price|distance_to_center|not_first_level|not_last_level|
+-----+------+-----+-----+------------+-------------+-----------+---------+------------------+---------------+--------------+
|    4|    13|    3| 85.0|        12.0|            3|          0|   1.07E7|25.259194068696385|              1|             1|
|    5|    17|   -1|32.44|         0.0|            0|          2|3214804.0| 33.48720302595378|              1|             1|
|    3|    17|    2| 62.4|         9.7|            0|          0|   1.11E7| 17.50983382864081|              1|             1|
|    2|    17|    1|36.99|       11.72|            4|          2|4296056.0|25.435581975122673|              1|             1|
|    5|     5|    3| 66.1|         9.0|            2|          0|3899000.0|  33.2971381244445|              1|        

Среди столбцов датафрейма можно увидеть колонку building_type — тип строения. Значения, которые она принимает, показаны ниже в ячейке. Эта колонка — категориальный признак. А как мы знаем, прежде чем начать обучать модель, нужно трансформировать категориальные признаки в векторы или в колонки бинарных признаков.

Значения building_type: 

0 — unknown (неизвестный).

1 — other (другой).

2 — panel (панельный).

3 — monolithic (монолитный).

4 — brick (кирпичный).

5 — blocky (блочный).

6 — wooden (деревянный).



Векторизуем категориальный признак с помощью OneHotEncoder.

In [ ]:
from pyspark import mllib
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
features_inp = ["building_type"] #Имя категориальной колонки

features_out = ["building_type_vec"] #Имя новой колонки, которая получится после преобразований

encoder = OneHotEncoder(inputCols = features_inp, outputCols = features_out) #Cоздадим объект encoder

Сначала при помощи метода fit() обучим encoder, в который «скормим» наш датафрейм.

Затем, воспользовавшись обученным encoder, преобразуем датафрейм методом transform() и удалим старую непреобразованную колонку при помощи drop().

In [ ]:
fitted_encoder = encoder.fit(realty_S3) 
realty_S4 = fitted_encoder.transform(realty_S3).drop("building_type")

Видим, что колонка преобразовалась в вектор (building_type_vec).

In [ ]:
realty_S4.show(5)

+-----+------+-----+-----+------------+-----------+---------+------------------+---------------+--------------+-----------------+
|level|levels|rooms| area|kitchen_area|object_type|    price|distance_to_center|not_first_level|not_last_level|building_type_vec|
+-----+------+-----+-----+------------+-----------+---------+------------------+---------------+--------------+-----------------+
|    4|    13|    3| 85.0|        12.0|          0|   1.07E7|25.259194068696385|              1|             1|    (6,[3],[1.0])|
|    5|    17|   -1|32.44|         0.0|          2|3214804.0| 33.48720302595378|              1|             1|    (6,[0],[1.0])|
|    3|    17|    2| 62.4|         9.7|          0|   1.11E7| 17.50983382864081|              1|             1|    (6,[0],[1.0])|
|    2|    17|    1|36.99|       11.72|          2|4296056.0|25.435581975122673|              1|             1|    (6,[4],[1.0])|
|    5|     5|    3| 66.1|         9.0|          0|3899000.0|  33.2971381244445|          

###Немного о векторах в библиотеке PySpark ML

In [ ]:
from pyspark.ml.linalg import Vector, Vectors

DenseVector (плотный вектор) — обычный вектор, задавая который нужно прописать значение каждой его компоненты.



In [ ]:
vec1 = Vectors.dense([1, 2, 3]) #Создаём плотный вектор
vec1

DenseVector([1.0, 2.0, 3.0])

SparseVector (разреженный вектор) — вектор, который состоит преимущественно из нулей. Только несколько его координат имеют отличные от нуля значения.

Задавая такой вектор, нужно указать кортеж, в котором перечисляется размерность, список индексов, где есть ненулевые значения, и сами ненулевые значения. 

Запись, показанная в ячейке, создаст вектор, состоящий из шести компонент. На позиции 1 и 4 будут находиться числа 2 и –3,5.


In [ ]:
vec2 = Vectors.sparse(6, [1, 4], [2.0, -3.5]) #Создаём разреженный вектор
vec2

SparseVector(6, {1: 2.0, 4: -3.5})

Чтобы преобразовать разреженный вектор в плотный, нужно сначала преобразовать его в NumPy array методом toArray(), а затем обернуть в denseVector, как это показано в ячейке.

In [ ]:
Vectors.dense(vec2.toArray()) #Преобразуем разреженный вектор в плотный

DenseVector([0.0, 2.0, 0.0, 0.0, -3.5, 0.0])

### Обучение модели

In [ ]:
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.regression import LinearRegression

Выделим из датафрейма список фичей и столбец target.

In [ ]:
features = list(realty_S4.drop("price").columns) #Список колонок фичей

target = "price" #Колонка «таргет»

В отличие от знакомой вам библиотеки Sklearn, в PySpark ML перед обучением моделей необходимо объединять все колонки признаков в многомерные векторы. Провести векторизацию нам поможет объект класса VectorAssembler.

Векторизуем фичи с помощью VectorAssembler.

In [ ]:
vectorizer = VectorAssembler(inputCols = features, outputCol = "features_vec") 
#Создаём переменную vectorizer — объект класса VectorAssembler, цель которого — превратить колонки с фичами в векторы

realty_vectorised = vectorizer.transform(realty_S4)
#Трансформируем при помощи vectorizer наш датафрейм и сохраним его в переменную realty_vectorised

Как видим, все колонки фичей превратились в одну — features_vec.

In [ ]:
realty_vectorised.select("features_vec","price").show(5,100)

+---------------------------------------------------------------------------------+---------+
|                                                                     features_vec|    price|
+---------------------------------------------------------------------------------+---------+
|  [4.0,13.0,3.0,85.0,12.0,0.0,25.259194068696385,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]|   1.07E7|
|  [5.0,17.0,-1.0,32.44,0.0,2.0,33.48720302595378,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0]|3214804.0|
|    [3.0,17.0,2.0,62.4,9.7,0.0,17.50983382864081,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0]|   1.11E7|
|[2.0,17.0,1.0,36.99,11.72,2.0,25.435581975122673,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0]|4296056.0|
|          (15,[0,1,2,3,4,6,7,11],[5.0,5.0,3.0,66.1,9.0,33.2971381244445,1.0,1.0])|3899000.0|
+---------------------------------------------------------------------------------+---------+
only showing top 5 rows



Создадим модель, используя класс LinearRegression, куда в качестве параметров передадим имя колонки с вектором фичей (featuresCol = 'features_vec') и имя колонки, значение в которой хотим предсказывать (назовём её target), — labelCol = target.


In [ ]:
model = LinearRegression(featuresCol = 'features_vec', labelCol = target)
#Создаём объект model — экземпляр класса LinearRegression

Разбиваем датасет на обучающую и тестовую выборку, используя randomSplit:

80% строк — на обучающую выборку.

20% строк — на тестовую.

In [ ]:
realty_train, realty_test = realty_vectorised.randomSplit([0.8, 0.2])

Для обучения модели воспользуемся поиском по сетке гиперпараметров с кросс-валидацией. Создадим два объекта: саму сетку гиперпараметров и evaluator — объект, чьей задачей будет оценивать качество предсказания нашей модели.

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator

Определяем сетку гиперпараметров при помощи ParamGridBuilder.

In [ ]:
grid = ParamGridBuilder().addGrid(model.regParam, [0.5, 5]).addGrid(model.elasticNetParam, [0.01, 0.1]).build()
#Создана сетка гиперпараметров, в которой параметр regParam принимает значения 0.5, 5, а параметр elasticNetParam — значения 0.01, 0.1

Создаём объект eval — экземпляр класса RegressionEvaluator, куда в качестве параметров пишем имя столбцов с предсказанным и реальным значением.

По умолчанию для оценки качества модели RegressionEvaluator использует метрику MSE — среднеквадратичную ошибку.

In [ ]:
eval = RegressionEvaluator(predictionCol='prediction', labelCol = target)
#Столбец с предсказаниями будет иметь имя 'prediction'

Теперь создаём CV-объект — экземпляр класса CrossValidator, куда в качестве параметров передаём ранее определённую модель model, сетку гиперпараметров grid и метрику качества eval.

Для обучения модели, как и в Sklearn, будем использовать метод fit().

Нужно подождать, пока модель обучается.

In [ ]:
cv = CrossValidator(estimator = model, estimatorParamMaps = grid, evaluator = eval) #Создаём объект cv — экземпляр CrossValidator
cv_model = cv.fit(realty_train) #Используем метод fit для обучения модели, скормив ему обучающую выборку

Выберем лучшую модель, обученную при кросс-валидации, с помощью метода bestModel.

In [ ]:
best_model = cv_model.bestModel

Посмотрим гиперпараметры лучшей модели при помощи extractParamMap.

In [ ]:
best_model.extractParamMap()

{Param(parent='LinearRegression_7100a461b518', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LinearRegression_7100a461b518', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1,
 Param(parent='LinearRegression_7100a461b518', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber'): 1.35,
 Param(parent='LinearRegression_7100a461b518', name='featuresCol', doc='features column name.'): 'features_vec',
 Param(parent='LinearRegression_7100a461b518', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LinearRegression_7100a461b518', name='labelCol', doc='label column name.'): 'price',
 Param(parent='LinearRegression_7100a461b518', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber.'): 'squaredE

Чтобы было удобно применять обученную модель в разных кейсах, напишем функцию, которая принимает на вход параметры квартиры, а на выходе предсказывает её цену.

In [ ]:
def price_predictor(level, levels, rooms, area, kitchen_area, geo_lat, geo_lon, building_type, object_type):
  
  #Зададим список колонок
  columns = ["level", "levels", "rooms", "area", "kitchen_area", "geo_lat", "geo_lon", "building_type", "object_type"]
  
  #Представим входные параметры как список кортежей
  #Один кортеж — одна строка в создаваемом датафрейме
  values = [(level, levels, rooms, area, kitchen_area, geo_lat, geo_lon, building_type, object_type)]

  #При помощи функции createDataframe создаём spark Dataframe, в качестве параметров передаём список имён столбцов и данные для таблицы
  df = spark.createDataFrame(values,columns) 

  #Добавляем в датафрейм новые фичи, которые мы обсуждали на этапе data preparation. Как видим по коду, написанному ниже, все преобразования данных можно делать цепочкой
  df_transformed = df.withColumn("distance_to_center",coord_to_distance(col("geo_lat"),col("geo_lon"))).drop("geo_lat","geo_lon") \
                      .withColumn("not_first_level",when(col("level") == 1,0).otherwise(1)) \
                      .withColumn("not_last_level",when(col("level") == col("levels"),0).otherwise(1)) \

  #Векторизуем категориальный признак — тип строения
  df_transformed_vec = fitted_encoder.transform(df_transformed)

  #Затем превратим в вектор все фичи нашего набора данных             
  df_vectorised = vectorizer.transform(df_transformed_vec)

  #Скормим полученный вектор фичей нашей обученной модели в метод transform() и на выходе получим цену квартиры
  return best_model.transform(df_vectorised.select("features_vec")).select("prediction")

In [ ]:
#Однушка в Королёве в кирпичном доме, вторичка
price_predictor(level = 21, levels = 25, rooms = 1, area = 43.0, kitchen_area = 10.0, geo_lat = 55.917188, geo_lon = 37.804221, building_type = 4, object_type = 2).show()

+-----------------+
|       prediction|
+-----------------+
|5860674.905795745|
+-----------------+



In [ ]:
#Трёшка в жилом небоскрёбе рядом с МЦК, вторичка
price_predictor(level = 50, levels = 56, rooms = 3, area = 120.0, kitchen_area = 20.0, geo_lat = 55.835395, geo_lon = 37.658311, building_type = 4, object_type = 2).show()

+-------------------+
|         prediction|
+-------------------+
|1.840236967033116E7|
+-------------------+

